In [35]:
import copy
import random
import sys
import numpy as np
import time
# path="D:\PythonProject\CARP\CARP_samples\egl2\egl-g1-A.dat"
# path="D:\PythonProject\CARP\sample.dat"
start=time.time()
path="D:\PythonProject\CARP\CARP_samples\\val7A.dat"
with open(path,"r") as f:
    content=f.readlines()
# print (content)
NAME=content[0].split(":")[1]
VERTICES_NUMBER=(int)(content[1].split(":")[1])
DEPOT=(int)(content[2].split(":")[1])
REQUIRED_EDGES=(int)(content[3].split(":")[1])
NON_REQUIRED_EDGES=(int)(content[4].split(":")[1])
VEHICLES=(int)(content[5].split(":")[1])
CAPACITY=(int)(content[6].split(":")[1])
RequiredEdgeCost=(int)(content[7].split(":")[1])
EdgeNumber=REQUIRED_EDGES+NON_REQUIRED_EDGES
class edge:
    def __init__(self,node1,node2,cost,demand,index):
        self.node1=node1
        self.node2=node2
        self.cost=cost
        self.demand=demand
        self.index=index

# class node:
#     visited
#     children
edges=[]
nodeToEdge={}
nodeToIndex={}
for i in range(EdgeNumber):
    content[9+i]=content[9+i].split()
    Edge=edge((int)(content[9+i][0])-1,(int)(content[9+i][1])-1,(int)(content[9+i][2]),(int)(content[9+i][3]),i)
    nodeToEdge[(Edge.node1,Edge.node2)]=Edge
    nodeToEdge[(Edge.node2,Edge.node1)]=Edge
    edges.append(Edge)
for i in range(EdgeNumber):
    print(edges[i].node1+1,end=" ")
    print(edges[i].node2+1,end=" ")
    print(edges[i].cost,end=" ")
    print(edges[i].demand)
# print(nodeToEdge.get((0,1)))
# print(nodeToEdge.get((0,1)).node1)
# nodeToIndex

1 2 2 3
1 6 3 8
1 8 4 9
1 10 8 12
1 11 3 12
1 33 4 9
1 35 4 12
1 40 4 10
2 3 5 9
2 6 2 8
2 36 3 12
3 4 2 5
3 7 2 3
3 37 4 6
4 5 3 7
4 38 3 4
5 39 2 6
6 7 4 7
6 12 2 4
7 13 1 3
8 9 5 12
8 14 7 10
8 40 5 8
9 10 2 6
9 15 2 9
10 16 4 8
11 12 7 9
11 16 3 10
12 13 6 8
12 17 4 8
13 17 2 6
13 18 2 5
13 19 6 12
14 15 4 7
15 16 3 8
17 18 5 8
17 20 6 12
18 19 4 14
18 22 6 13
20 21 2 3
20 23 3 8
21 22 1 4
21 24 3 6
22 25 1 3
23 24 2 7
24 25 2 5
26 27 4 8
26 33 5 14
26 34 5 16
27 28 3 11
27 34 3 9
28 35 10 15
29 30 3 12
29 36 4 9
30 31 3 10
30 37 3 5
31 32 5 12
31 37 5 7
31 38 4 9
32 39 2 8
33 34 3 8
34 35 6 8
35 36 6 14
36 37 4 13
37 38 2 4
38 39 7 9


In [36]:
MAX = 1<<32-1
distance_matrix=np.ones((VERTICES_NUMBER,VERTICES_NUMBER),dtype=int)*MAX
for Edge in edges:
    node1=Edge.node1
    node2=Edge.node2
    distance_matrix[node1][node2]=Edge.cost
    distance_matrix[node2][node1]=Edge.cost

def dijkstra(matrix, start_node):
    matrix_length = len(matrix)  # 矩阵一维数组的长度，即节点的个数

    used_node = [False] * matrix_length  # 访问过的节点数组

    distance = [MAX] * matrix_length  # 最短路径距离数组

    distance[start_node] = 0  # 初始化，将起始节点的最短路径修改成0

    # 将访问节点中未访问的个数作为循环值，其实也可以用个点长度代替。
    while used_node.count(False):
        min_value = MAX
        min_value_index = -1

        # 在最短路径节点中找到最小值，已经访问过的不在参与循环。
        # 得到最小值下标，每循环一次肯定有一个最小值
        for index in range(matrix_length):
            if not used_node[index] and distance[index] <= min_value:
                min_value = distance[index]
                min_value_index = index

        # 将访问节点数组对应的值修改成True，标志其已经访问过了
        used_node[min_value_index] = True

        # 更新distance数组。
        # 以B点为例：distance[x] 起始点达到B点的距离。
        # distance[min_value_index] + matrix[min_value_index][index] 是起始点经过某点达到B点的距离，比较两个值，取较小的那个。
        for index in range(matrix_length):
            distance[index] = min(distance[index], distance[min_value_index] + matrix[min_value_index][index])

    return distance
# dijkstra(distance_matrix,0)
Shortest_Distance=[]
for i in range(VERTICES_NUMBER):
    Shortest_Distance.append(dijkstra(distance_matrix,i))

In [37]:
import copy
#Path Scanning
# print(VERTICES_NUMBER)
# print(Shortest_Distance)
#Rule:
# 1) maximize the distance from the task to the depot;
# 2) minimize the distance from the task to the depot;
# 3) maximize the term dem(t)/sc(t), where dem(t) and sc(t) are demand and serving cost of task t, respectively;
# 4) minimize the term dem(t)/sc(t);
# 5) use rule 1) if the vehicle is less than half- full, otherwise use rule 2)
#better returns whether u1 is better than u2
def better(u1,u2,load,rule):
    node1=u1.node1
    node2=u2.node1
    distance1=Shortest_Distance[node1][DEPOT-1]
    distance2=Shortest_Distance[node2][DEPOT-1]
    if rule==1:
        if distance1 >=distance2:
            return True
        else:
            return False
    elif rule==2:
        if distance1 <=distance2:
            return True
        else:
            return False
    elif rule==3:
        if u1.demand/u1.cost >=u2.demand/u2.cost:
            return True
        else:
            return False
    elif rule==4:
         if u1.demand/u1.cost <=u2.demand/u2.cost:
            return True
         else:
            return False
    else:
        if load<=CAPACITY/2:
            return True
        else:
            return False

demand_arc=[]
count=0
for u in edges:
    if u.demand!=0:
        nodeToIndex[(u.node1,u.node2)]=2*count
        nodeToIndex[(u.node2,u.node1)]=2*count+1
        demand_arc.append((u.node1,u.node2))
        demand_arc.append((u.node2,u.node1))
        count=count+1
#需求边集free

def path_scanning(demand_arc,rule,VEHICLES):
    free=copy.deepcopy(demand_arc)
    k=0
    routines=[]
    costs=[]
    while len(free)!=0 and k<VEHICLES:
        routine=[]
        k=k+1
        load=0
        cost=0
        i=DEPOT-1
        d=0
        # d=MAX
        while len(free)!=0 and d!=MAX :
            d=MAX
            for item in free :
                u=nodeToEdge.get(item)
                if load+u.demand<=CAPACITY:
                    distance=Shortest_Distance[i][item[0]]
                    if distance<d:
                        d=distance
                        chosen_arc=item
                    elif distance==d and better(u,nodeToEdge.get(chosen_arc),load,rule):
                        chosen_arc=item

            if d!=MAX:
                chosen_edge=nodeToEdge.get(chosen_arc)
                routine.append(chosen_arc)
                free.remove(chosen_arc)
                free.remove((chosen_arc[1],chosen_arc[0]))
                load=load+chosen_edge.demand
                cost=cost+d+chosen_edge.cost
                i=chosen_arc[1]
        cost=cost+Shortest_Distance[i][DEPOT-1]
        routines.append(routine)
        costs.append(cost)
    # if len(free)!=0:
    #     invalid_routine.append(routines)
    # else:
    #     valid_routine.append(routines)
    return routines, sum(costs)

# print(routines)
# print(sum(costs))

Solution, Cost = [], MAX
for i in range(5):
    solution,cost=path_scanning(demand_arc,i,VEHICLES)
    if cost<Cost:
        Solution=solution
        Cost=cost
print("s",end=" ")
for i in range(len(Solution)):
    print(0,end=",")
    for j in range(len(Solution[i])):
        Solution[i][j]=(Solution[i][j][0]+1,Solution[i][j][1]+1)
        print(Solution[i][j],end=",")
    if(i==len(Solution)-1 and j==len(Solution[i])-1):
        print(0,end="")
    else:
        print(0,end=",")
print()
print("q",end=" ")
print(Cost)

for i in range(EdgeNumber):
    print(edges[i].node1,end=" ")
    print(edges[i].node2,end=" ")
    print(edges[i].cost,end=" ")
    print(edges[i].demand)

# print(NAME)
# print(VERTICES_NUMBER)
# print(DEPOT)
# print(REQUIRED_EDGES)
# print(NON_REQUIRED_EDGES)
# print(VEHICLES)
# print(CAPACITY)
# print(RequiredEdgeCost)

# content[1]

s 0,(1, 10),(10, 16),(16, 15),(15, 14),(14, 8),(8, 40),(40, 1),(1, 2),(2, 3),(3, 37),(37, 31),(31, 38),(38, 39),(39, 5),(5, 4),(4, 38),(38, 37),(37, 30),(30, 31),(31, 32),(32, 39),(4, 3),(3, 7),(7, 6),(6, 12),(12, 11),(11, 16),0,0,(1, 33),(33, 34),(34, 35),(35, 28),(28, 27),(27, 26),(26, 33),(34, 27),(34, 26),(1, 8),(8, 9),(9, 10),(9, 15),(11, 1),(1, 6),(6, 2),(2, 36),(36, 29),(29, 30),(7, 13),0,0,(1, 35),(35, 36),(36, 37),(13, 12),(12, 17),(17, 18),(18, 22),(22, 25),(25, 24),(24, 21),(21, 20),(20, 17),(17, 13),(13, 19),(19, 18),(18, 13),(20, 23),(23, 24),(22, 21),0
q 335
0 1 2 3
0 5 3 8
0 7 4 9
0 9 8 12
0 10 3 12
0 32 4 9
0 34 4 12
0 39 4 10
1 2 5 9
1 5 2 8
1 35 3 12
2 3 2 5
2 6 2 3
2 36 4 6
3 4 3 7
3 37 3 4
4 38 2 6
5 6 4 7
5 11 2 4
6 12 1 3
7 8 5 12
7 13 7 10
7 39 5 8
8 9 2 6
8 14 2 9
9 15 4 8
10 11 7 9
10 15 3 10
11 12 6 8
11 16 4 8
12 16 2 6
12 17 2 5
12 18 6 12
13 14 4 7
14 15 3 8
16 17 5 8
16 19 6 12
17 18 4 14
17 21 6 13
19 20 2 3
19 22 3 8
20 21 1 4
20 23 3 6
21 24 1 3
22 23 2

In [38]:
# path_scanning(demand_arc,1,VEHICLES)

In [39]:
def fitness(routines):
    cost=0
    for i in range(len(routines)):
        start=DEPOT-1
        for j in range(len(routines[i])):
            begin=routines[i][j][0]
            end=routines[i][j][1]
            # print(begin, end)
            cost=cost+Shortest_Distance[start][begin]+nodeToEdge.get((begin,end)).cost
            start=end
        cost=cost+Shortest_Distance[end][DEPOT-1]
    return cost
# road=[[(1, 8), (8, 3), (3, 4), (4, 11), (11, 12), (10, 1)],
#  [(1, 11), (11, 6), (6, 4), (4, 1), (9, 1)],
#  [(1, 2), (2, 4), (4, 7), (7, 2), (2, 5), (5, 1)],
#  [(6, 5), (5, 7), (2, 3), (3, 9), (9, 8), (8, 12), (12, 10), (10, 9)]]
# fitness(road)


In [40]:
import random
def random_pick():
    free=copy.deepcopy(demand_arc)
    random.shuffle(free)
    chosen=np.zeros(len(free))
    k=0
    routines=[]
    costs=[]
    while len(free)!=0 and k<VEHICLES:
        routine=[]
        k=k+1
        load=0
        cost=0
        i=DEPOT-1
        # d=MAX
        for item in free:
            index=nodeToIndex.get(item)
            if chosen[index]==0:
                u=nodeToEdge.get(item)
                if load+u.demand<=CAPACITY:
                    chosen_arc=item
                    chosen_edge=nodeToEdge.get(chosen_arc)
                    routine.append(chosen_arc)
                    load=load+chosen_edge.demand
                    cost=cost+Shortest_Distance[i][item[0]]+chosen_edge.cost
                    i=chosen_arc[1]
                    chosen[nodeToIndex.get(item)]=1
                    chosen[nodeToIndex.get((item[1],item[0]))]=1
        cost=cost+Shortest_Distance[i][DEPOT-1]
        routines.append(routine)
        costs.append(cost)
    # if len(free)!=0:
    #     invalid_routine.append(routines)
    # else:
    #     valid_routine.append(routines)
    return routines, sum(costs)
random_pick()

([[(39, 0),
   (1, 0),
   (8, 14),
   (17, 16),
   (25, 33),
   (12, 11),
   (13, 14),
   (2, 36),
   (19, 20),
   (24, 23),
   (23, 20),
   (16, 11),
   (3, 2),
   (21, 20),
   (29, 28),
   (27, 26),
   (8, 7),
   (6, 5),
   (32, 33),
   (39, 7),
   (17, 18),
   (22, 19),
   (12, 18),
   (5, 0)],
  [(17, 21),
   (35, 34),
   (12, 6),
   (1, 2),
   (3, 37),
   (32, 0),
   (0, 9),
   (24, 21),
   (0, 7),
   (16, 19),
   (16, 12),
   (15, 10),
   (0, 34),
   (22, 23),
   (28, 35),
   (30, 37),
   (35, 36),
   (31, 38),
   (12, 17),
   (9, 15),
   (9, 8),
   (31, 30),
   (11, 5),
   (6, 2)],
  [(34, 33),
   (37, 36),
   (30, 29),
   (1, 5),
   (26, 33),
   (30, 36),
   (10, 11),
   (14, 15),
   (29, 36),
   (4, 38),
   (34, 27),
   (35, 1),
   (13, 7),
   (32, 25),
   (4, 3),
   (26, 25),
   (37, 38),
   (0, 10)]],
 1078)

In [41]:
def is_Valid(routines):
    tasks = []
    for r in routines:
        for t in r:
            tasks.append(t)
            tasks.append((t[1], t[0]))
    task_set = set(tasks)
    return len(tasks) == len(task_set) and len(tasks) / 2 == REQUIRED_EDGES

valid_routine=[]
invalid_routine=[]
def init_population(pop_size):
    """
    Randomly initialize a population for genetic algorithm
        pop_size  :  Number of individuals in population
        gene_pool   :  List of possible values for individuals
        state_length:  The length of each individual
    """
    # global valid_routine
    # global invalid_routine
    population = []
    for i in range(5):
        solution,cost=path_scanning(demand_arc,i,VEHICLES)
        population.append(solution)
        # if is_Valid(solution):
        #     valid_routine.append(solution)
        # else:
        #     invalid_routine.append(solution)
    # for _ in range(pop_size-5):
    #     solution,cost=random_pick()
    #     if is_Valid(solution):
    #         valid_routine.append(solution)
    #     else:
    #         invalid_routine.append(solution)
    while len(population)<pop_size:
        solution,cost=random_pick()
        if is_Valid(solution):
            population.append(solution)
    # population=valid_routine+invalid_routine
    return population

In [42]:
population=init_population(20)
print(population)
print(len(population))
minimum=sys.maxsize
for item in population:
    f=fitness(item)
    print(f)
    if f<minimum:
        minimum=f
print(population[2])
minimum

[[[(0, 39), (39, 7), (7, 13), (13, 14), (14, 15), (15, 10), (10, 11), (11, 16), (16, 19), (19, 22), (22, 23), (23, 24), (24, 21), (21, 17), (17, 12), (12, 6), (6, 2), (2, 1), (1, 0), (0, 5), (5, 1), (1, 35), (35, 28), (28, 29), (29, 30)], [(0, 34), (34, 35), (35, 36), (36, 37), (37, 38), (38, 31), (31, 30), (30, 37), (37, 3), (3, 4), (4, 38), (3, 2), (2, 36), (36, 29), (30, 36), (6, 5), (5, 11), (11, 12), (12, 16), (16, 17), (17, 18), (18, 12), (0, 7), (0, 32)], [(0, 10), (15, 9), (9, 8), (8, 14), (8, 7), (0, 9), (34, 33), (33, 32), (32, 25), (25, 33), (33, 26), (26, 25), (26, 27), (27, 34), (19, 20), (20, 21), (20, 23)]], [[(0, 39), (39, 7), (7, 13), (13, 14), (14, 15), (15, 9), (9, 8), (8, 14), (8, 7), (7, 0), (0, 34), (34, 27), (27, 26), (26, 33), (33, 25), (25, 32), (32, 33), (33, 34), (34, 35), (1, 0), (6, 12)], [(0, 32), (0, 10), (10, 15), (10, 11), (11, 16), (16, 19), (19, 22), (22, 23), (23, 24), (24, 21), (21, 20), (20, 23), (20, 19), (21, 17), (17, 18), (18, 12), (12, 17), (1

335

In [43]:
def is_overloaded(route):
    cnt = 0
    for item in route:
        u = nodeToEdge.get(item)
        cnt = cnt + u.demand
    if cnt <= CAPACITY:
        return False
    else:
        return True


def single_insertion(routine):
    minimum = fitness(routine)
    chosen = routine
    routines = copy.deepcopy(routine)
    route=[]
    while len(route)==0:
     route=random.choice(routines)
    task = random.choice(route)
    task2=(task[1],task[0])
    route.remove(task)
    for i in range(len(routines)):
        for j in range(len(routines[i])):
            routines[i].insert(j, task)
            if not is_overloaded(routines[i]) :
                f = fitness(routines)
                if f < minimum:
                    chosen = copy.deepcopy(routines)
                    minimum =f
            routines[i].remove(task)

    for i in range(len(routines)):
        for j in range(len(routines[i])):
            routines[i].insert(j, task2)
            if not is_overloaded(routines[i]) :
                f = fitness(routines)
                if f < minimum:
                    chosen = copy.deepcopy(routines)
                    minimum =f
            routines[i].remove(task2)

    return chosen
# print(population[2])
# print(fitness(population[2]))
# print(fitness(single_insertion(population[2])))

In [44]:
def double_insertion(routine):
    minimum = fitness(routine)
    chosen = routine
    routines = copy.deepcopy(routine)
    route = []
    while len(route) < 2:
        route = random.choice(routines)
    index = random.randint(0,len(route)-2)
    task = [route[index],route[index+1]]
    # print(task)
    task2 = [(task[1][1],task[1][0]),(task[0][1],task[0][0])]
    # print(task2)
    route.remove(task[0])
    route.remove(task[1])
    for i in range(len(routines)):
        for j in range(len(routines[i])):
            routines[i].insert(j, task[0])
            routines[i].insert(j+1,task[1])
            if not is_overloaded(routines[i]):
                f = fitness(routines)
                # print(f)
                if f < minimum:
                    chosen = copy.deepcopy(routines)
                    minimum = f
            routines[i].remove(task[0])
            routines[i].remove(task[1])

    for i in range(len(routines)):
        for j in range(len(routines[i])):
            routines[i].insert(j, task2[0])
            routines[i].insert(j+1,task2[1])
            if not is_overloaded(routines[i]):
                f = fitness(routines)
                # print(f)
                if f < minimum:
                    chosen = copy.deepcopy(routines)
                    minimum = f
            routines[i].remove(task2[0])
            routines[i].remove(task2[1])
    return chosen


In [45]:
# def swap(routine):
#     minimum = fitness(routine)
#     chosen = routine
#     for i in range(10):
#         routines = copy.deepcopy(routine)
#         route=[]
#         while len(route)==0:
#          route=random.choice(routines)
#         task = random.choice(route)
#         print(task)
#         # task2=(task[1],task[0])
#         # route.remove(task)
#         route2=[]
#         while len(route2)==0:
#          route2=random.choice(routines)
#         task2 = random.choice(route)
#         print(task2)
#
#         mid=task
#         task=task2
#         task2=mid
#         print(routines)
#
#         f=fitness(routines)
#         # print(f)
#         if f <minimum:
#             chosen=routines
#     return chosen

In [46]:
# print(population[0])
# print(swap(population[0]))

In [47]:
#输入一个方案，任选两辆车的子图进行PS，然后返回新的方案
def Merge_Split(routine):
    routines = copy.deepcopy(routine)
    #随机选取两个小车
    route1 = random.choice(routines)
    routines.remove(route1)
    route2 = random.choice(routines)
    routines.remove(route2)
    #对两个小车构成的route进行PS
    route = []
    for item in route1:
        route.append(item)
        route.append((item[1], item[0]))
    for item in route2:
        route.append(item)
        route.append((item[1], item[0]))
    #选择5个规则下面最好的
    minimum = sys.maxsize
    chosen_routine = None
    for i in range(5):
        solution, cost = path_scanning(route, i, 2)
        if cost < minimum:
            chosen_routine = solution
            minimum = cost
    routines=routines+chosen_routine
    if fitness(routines)<fitness(routine):
        return routines
    else:
        return routine


In [48]:
# print(fitness(population[0]))
# print(fitness(Merge_Split(population[0])))
N=20000
# print(population)
for i in range(N):
    individual=random.choice(population)
    new_individual=single_insertion(individual)
    new_individual=double_insertion(new_individual)
    # print(new_individual)
    new_individual=Merge_Split(new_individual)
    if is_Valid(new_individual) :
        population.remove(individual)
        population.append(new_individual)
    # population.append(single_insertion(individual))
# print(len(population))
minimum=sys.maxsize
print(population)
for item in population:
    if is_Valid(item):
        f=fitness(item)
        print(f)
        if f<minimum:
            minimum=f
# print()
# print(len(population))
minimum

[[[(0, 34), (34, 27), (27, 26), (25, 32), (32, 0), (0, 7), (7, 8), (8, 9), (9, 15), (15, 14), (14, 13), (13, 7), (7, 39), (39, 0), (0, 9), (8, 14), (15, 10)], [(0, 1), (1, 2), (2, 3), (3, 4), (4, 38), (38, 31), (31, 30), (30, 37), (37, 38), (3, 37), (37, 36), (36, 30), (30, 29), (29, 36), (36, 2), (2, 6), (6, 12), (12, 18), (18, 17), (17, 21), (21, 20), (20, 23), (23, 24), (24, 21), (23, 22), (22, 19), (20, 19), (19, 16)], [(0, 5), (5, 6), (12, 17), (17, 16), (16, 12), (12, 11), (11, 16), (11, 5), (5, 1), (1, 35), (35, 28), (28, 29), (36, 35), (35, 34), (34, 33), (33, 25), (25, 26), (26, 33), (33, 32), (0, 10), (10, 11)]], [[(0, 7), (7, 13), (13, 14), (14, 8), (14, 15), (15, 9), (9, 0), (0, 10), (10, 11), (11, 12), (0, 32), (32, 25), (25, 26), (25, 33), (33, 26), (26, 27), (27, 34), (34, 33), (33, 32)], [(0, 39), (39, 7), (7, 8), (8, 9), (15, 10), (0, 1), (1, 5), (5, 6), (6, 12), (12, 16), (16, 19), (20, 19), (19, 22), (22, 23), (23, 24), (24, 21), (21, 20), (20, 23), (21, 17), (17, 18

279

In [49]:
#1137630

In [50]:
#1110534, 3675, 5359, 316 , 275 ,278, 404,283

In [51]:
#1110943, 3788 ,5303